In [14]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import time
import schedule

def tyres():
    url = 'https://www.national.co.uk/price-comparison' # set the target website
    page = requests.get(url) #get connection request
    
    soup = BeautifulSoup(page.content, 'html.parser') # start of our soup

    tbl = soup.find('table', {'class':'table table-striped table-bordered'}) # reaching target table

    hdrs = []

    #for loop below grabs table header names 
    for i in tbl.find_all('th'):
        title = i.text.strip()
        hdrs.append(title)
    
    df = pd.DataFrame(columns = hdrs)
    
    #for loop below fetches data from the table
    for row in tbl.find_all('tr')[1:]:
        data = row.find_all('td')
        row_data = [td.text.strip() for td in data]
        length = len(df)
        df.loc[length] = row_data

    #now we drop unnecessary columns
    df = df.drop(['MY TYRES PRICE', 'KWIK FIT PRICE', 'ATS PRICE'], axis =1)

    #removing any empty space from data
    #split data from the first column into multiple columns and assign this to sep DF
    df['PRODUCT DESCRIPTION'].str.strip()

    sep = df['PRODUCT DESCRIPTION'].str.split(' ', expand=True)

    #name the columns of a new DF
    sep.columns = ['1','2','3','4','5','6','7']

    #drop empty column and combine tyre pattern and size columns into one 
    sep.drop(['7'], axis=1)
    sep['x'] = sep['2']+ ' '+sep['3']+' '+sep['4']

    #from sep DF assign column 1 to a brand column in our main DF. Then repeat for pattern and size column
    df['Brand'] = sep['1']
    df['pattern & size'] = sep['x'] 
    df = df.drop(['PRODUCT DESCRIPTION'], axis=1) # drop unnecessary column
    df['url'] = url #add a colum with a url of scraped website

    csvFile = df.to_csv('newFile.csv', index=False) # write to csv
    
    return page, csvFile

# code below schedules running the code at a certain time to avoid accidental DDOS
schedule.every().day.at("12:00").do(tyres)
while True:
    schedule.run_pending()
    time.sleep(60)
